TODO:

1. Brief intro section on what LSTMs are
2. Go through coding up the math, step by step
3. Go through the theory of how data must flow through them
4. Go through the classes

5. Bonus: switch to autograd and go over variants such as GRUs

Goal: provide a tutorial that contains, side-by-side:

* Visual explanations of what is going on in an LSTM-based network
* Math explaining why what is going on is working
* Code implementing the math

# What LSTMs are

* LSTMs are used to model sequences, one sequence element at a time
* Any time the input is a sequence or the output is a sequence, an LSTM can be used

## Applications

* Generating the next character in text
    * Input is a corpus of text and (optionally) the sequence of text seen so far.
    * Output is the predicted next character
* An important component of sequence-to-sequence modeling, used in language translation
    * Input is a sequence of text, and the output is the sequence in another language.
* Image captioning
    * Input is a an image, output is a sequence of text describing the image.

# What they are

Here we'll cover how LSTMs model sequences.

To understand the forward pass of LSTMs, check out beautiful, comprehensive, and very clear blog post from Chris Olah: http://colah.github.io/posts/2015-08-Understanding-LSTMs/

In my opinion, to truly understand what is going on, it is necessary to see **visuals, math, and code** (or at least pseudo-code) all side by side - when I was first learning about neural nets, I remember reading many blog posts like Chris' that contained beautiful visuals, but it wasn't always clear to me how you'd translate those into code. 

## LSTMs, step by detailed step

Let's say we have a vocabulary of 60 characters, and an LSTM layer with 100 nodes. What actually goes on in the forward pass?

### Step 1

First: the input vector of length 60 is passed in, along with the cell hidden state of length 100.

$X_t$ and $h_{t-1}$

**Code:** Since a natural way to represent our data is as rows, `X` will be a 1x60 vector, and `H` will be a 1x100 vector. We'll use `np.column_stack` to combine these into a 1x160 vector we'll denote `Z`.

`z = np.column_stack((x,h))`

### Step 2

Then, these are concatenated into a vector of length 160, and multiplied together and fed through a sigmoid to create a forget gate $f_t$.

$f_t = \sigma(W_f * [X_t, h_{t-1}])$

[insert image]

**Code:** `f = sigmoid(np.dot(W_f, z))`, where $W_f$ is a 160x100 matrix.

### Step 3:

Similarly, an input gate is used to decide what of the current input to use: 

$i_t = \sigma(W_i * [X_t, h_{t-1}])$.

[visual here]

**Code:** `i = sigmoid(np.dot(W_i, z))`, where $W_i$ is a 160x100 matrix.

### Step 4:

Cell state computation

$\tilde{C_t} = tanh(W_c * [X_t, h_{t-1}])$.

[visual here]

**Code:** `C_tilde = tanh(np.dot(W_C, z))`, where $W_c$ is a 160x100 matrix.

### Step 5:

Compute $ C_t = f_t * C_{t-1} + i_t * \tilde{C_t} $

[visual here]

`C = f * C_prev + i * C_tilde`

### Step 6:

Compute $o_t = \sigma(W_o * [X_t, h_{t-1}])$

[visual here]

`o = sigmoid(np.dot(W_o, z))`

Note: $W_o$ has dimension 160 x 100.

### Step 7:

Compute $h_t = o_t * tanh(C_t)$

`h = o * tanh(C)`

[visual here]

### Step 8:

Compute $ V_t = W_V * h_t $ 

[visual here]

**Code:** `v = np.dot(W_v, h) + b_v`

Note: $W_v$ has dimension 100 x 60.

## Loss

This `v` is the output of the LSTM cell - a vector the same size as the input that contains predictions of what the next character will be. 

These are then compared with some other vector `y` that contains the "right answer". 

The loss is computed as: $L = (y-P) ^ 2$ - in code this would simply be `L = (y-P) ** 2`. 

The gradient of the loss with respect to the prediction therefore would be $-(y-P)$ - in code `-1.0 * (y-P)` - which is enough to let us begin the backwards pass.

## Backwards pass

[Intro to backwards pass]

We'll call the three quantities we pass in `loss_grad`, `dh_next`, and `dC_next`.

## Step 2

Recall in our basic neural net, when we did $ W * B = C $, and we were calculating $ \frac{\partial{L}}{\partial{W}} $ and $ \frac{\partial{L}}{\partial{B}} $, using $ \frac{\partial{L}}{\partial{C}} $, it turned out that:

$$ \frac{\partial{C}}{\partial{W}} = B^T $$

and

$$ \frac{\partial{C}}{\partial{B}} = W^T $$

So that, for example, because of the chain rule,

$$ \frac{\partial{L}}{\partial{W}} = \frac{\partial{L}}{\partial{C}} * \frac{\partial{C}}{\partial{W}} = \frac{\partial{L}}{\partial{W}} = \frac{\partial{L}}{\partial{C}} * B^T $$



So, since $ V_t = W_v * h_t $, 

We have both:

$$ \frac{\partial{L}}{\partial{W}} = \frac{\partial{L}}{\partial{V}} * H^T $$

and

$$ \frac{\partial{L}}{\partial{H}} = \frac{\partial{L}}{\partial{V}} * W_v^T $$

In code, we write:

`W_v_deriv += np.dot(self.H.T, loss_grad)`

and 

`dh = np.dot(loss_grad, W_v.T)`

## Step 3

To the current value of $ \frac{\partial{L}}{\partial{H}} $, we add the gradient from the next time step.

$$ \frac{\partial{L}}{\partial{H}} = \frac{\partial{L}}{\partial{H}} + H_{grad} $$

**Code:** `dh += dh_next`

## Step 4

Next we want to compute

$$ \frac{\partial{L}}{\partial{O}} $$

We have both $h_t = o_t * tanh(C_t)$

and by the logic presented for step 2, we have:

$$ \frac{\partial{L}}{\partial{O}} = \frac{\partial{L}}{\partial{H}} * tanh(C_t) $$

**Code:** `do = dh * tanh(self.C)`

## Step 4.5 

Call $ W_o * [X_t, h_{t-1}] = p$. To continue "moving down the chain", we want to compute

$$ \frac{\partial{L}}{\partial{P}} $$ 

We know that since $ o_t = \sigma(p) $, so the derivative of this function is $ \sigma(p) * (1 - \sigma(p)) $. Thus

$$ \frac{\partial{L}}{\partial{P}} = \frac{\partial{L}}{\partial{O}} * \sigma(p) * (1 - \sigma(p)) $$

## Step 5

Similarly to step 2, we have

$ W_o * [X_t, h_{t-1}] = p $

We'll see this quantity $[X_t, h_{t-1}]$ quite a bit, so we'll call it $z_t$.

Since 

$ W_o * z_t = p $

We have both:

$$ \frac{\partial{L}}{\partial{z_t}} = \frac{\partial{L}}{\partial{P}} * W_o^T $$

(though as we shall see, this is just one component of $\frac{\partial{L}}{\partial{z_t}}$) - and also:

$$ \frac{\partial{L}}{\partial{W_o}} = \frac{\partial{L}}{\partial{P}} * z^T $$

## Step 6:

Next we want to compute $$ \frac{\partial{L}}{\partial{C_t}} $$, and we have 

$h_t = o_t * tanh(C_t)$

First, we receive $C_{grad}$ from the layer above, so we can start by initializing

$$ \frac{\partial{L}}{\partial{C_t}} = C_{grad} $$

First, set $ tanh(C_t) = D_t $. Then, using exactly the same logic as in Step 2:

$$ \frac{\partial{L}}{\partial{D_t}} = \frac{\partial{L}}{\partial{H_t}} * o_t $$.

Then, since $ D_t = tanh(C_t) $, the derivative of this will be the derivative of the `tanh` function evaluated at $tanh(C_t)$. So, applying the chain rule:

$$ \frac{\partial{L}}{\partial{C_t}} = \frac{\partial{L}}{\partial{D_t}} * \frac{\partial{D_t}}{\partial{C_t}} = \frac{\partial{L}}{\partial{H_t}} * o_t * tanh'(tanh(C_t)) $$

## Step 7

Next we want to compute $$ \frac{\partial{L}}{\partial{\tilde{C_t}}} $$

Since $C_t = \tilde{C_t} * i_t$, where this represents an element-wise multiplication, we have simply:

$ \frac{\partial{L}}{\partial{\tilde{C_t}}} = \frac{\partial{L}}{\partial{C}} * i_t $ 

## Step 7.5

Next we want to ultimately compute $$ \frac{\partial{L}}{\partial{W_c}} $$. 

Following Step 4, we'll first define an intermediate quantity $ E_t = W_c * [X_t, h_{t-1}]$, and compute $ \frac{\partial{L}}{\partial{E_t}} $.

We have $ \tilde{C_t} = tanh(E_t)$, so 

$$ \frac{\partial{L}}{\partial{E_t}} = \frac{\partial{L}}{\partial{C_t}} * tanh'(tanh(E_t)) $$

## Step 8

Next, we'll compute $$ \frac{\partial{L}}{\partial{W_c}} $$. Similarly to Step 5, since:

$$ E_t = W_c * z_t $$

We have both:

$$ \frac{\partial{L}}{\partial{W_c}} = \frac{\partial{L}}{\partial{E_t}} * z^T $$

(multiplying by the transpose of $z$ since these are again matrix multiplications), and 

$$ \frac{\partial{L}}{\partial{Z}} = \frac{\partial{L}}{\partial{E_t}} * W_c^T $$

## Step 9

Step 9 follows Step 7. We want to compute $$ \frac{\partial{L}}{\partial{\tilde{i_t}}} $$

Since $C_t = \tilde{C_t} * i_t$, where this represents an element-wise multiplication, we have simply:

$ \frac{\partial{L}}{\partial{\tilde{i_t}}} = \frac{\partial{L}}{\partial{C}} * \tilde{C_t} $.

## Step 9.5

Similarly, step 9.5 follows Step 7.5. We ultimately want to compute $$ \frac{\partial{L}}{\partial{W_i}} $$. 

Following Step 4, we'll first define an intermediate quantity $ J_t = W_i * [X_t, h_{t-1}]$, and compute $ \frac{\partial{L}}{\partial{J_t}} $.

We have $i_t  = \sigma(J_t)$, so 

$$ \frac{\partial{L}}{\partial{J_t}} = \frac{\partial{L}}{\partial{i_t}} * \sigma(J_t) * (1-\sigma(J_t)) $$

## Step 10

Step 10 is identical to Step 8: we can now compute $$ \frac{\partial{L}}{\partial{W_i}} $$. 
Similarly to Step 5, since:

$$ J_t = W_i * z_t $$

We have both:

$$ \frac{\partial{L}}{\partial{W_i}} = \frac{\partial{L}}{\partial{J_t}} * z^T $$

and, adding to rolling sum of $ \frac{\partial{L}}{\partial{Z}} $, 

$$ \frac{\partial{L}}{\partial{Z_t}} = \frac{\partial{L}}{\partial{J_t}} * W_i^t $$

## Step 11

Step 11 follows Step 7. We want to compute $$ \frac{\partial{L}}{\partial{\tilde{f_t}}} $$

Since $ C_t = \tilde{f_t} * C_{prev}$, where this represents an element-wise multiplication, we have simply:

$$ \frac{\partial{L}}{\partial{f_t}} = \frac{\partial{L}}{\partial{C_t}} * C_{prev} $$

## Step 11.5

Similarly, step 11.5 follows step 7.5. We want to ultimately compute $$ \frac{\partial{L}}{\partial{W_f}} $$. 

Following Step 4, we'll first define an intermediate quantity $ G_t = W_f * [X_t, h_{t-1}]$, and compute $ \frac{\partial{L}}{\partial{G_t}} $.

We have $ f_t = \sigma(G_t)$, so 

$$ \frac{\partial{L}}{\partial{G_t}} = \frac{\partial{L}}{\partial{f_t}} * \sigma(G_t) * (1-\sigma(G_t)) $$

## Step 12

Step 12 is identical to Steps 8 and 10: we can now compute $$ \frac{\partial{L}}{\partial{W_i}} $$. 

Similarly to Step 5, since:

$$ G_t = W_f * z_t $$

We have both:

$$ \frac{\partial{L}}{\partial{W_f}} = \frac{\partial{L}}{\partial{G_t}} * z^T $$

and, adding to rolling sum of $ \frac{\partial{L}}{\partial{Z_t}} $, 

$$ \frac{\partial{L}}{\partial{Z_t}} = \frac{\partial{L}}{\partial{G_t}} * W_f^t $$

## Step 13

Step 13 follows steps 11 and 7. We want to compute $$ \frac{\partial{L}}{\partial{C_{prev}}} $$

Since $ C_t = f_t * C_{prev}$, where this represents an element-wise multiplication, we have simply:

$$ \frac{\partial{L}}{\partial{C_{prev}}} = \frac{\partial{L}}{\partial{C_t}} * f_t $$

## Step 14

Luckily, this isn't really a step, since we've alread computed the four components of 

$$ \frac{\partial{L}}{\partial{Z_t}} = \frac{\partial{L}}{\partial{G_t}} * W_f^t + \frac{\partial{L}}{\partial{J_t}} * W_i^t + \frac{\partial{L}}{\partial{E_t}} * W_c^T + \frac{\partial{L}}{\partial{P}} * W_o^T $$

## Step 15

On the forward pass, $Z_t$ was simply made up of $X_t$ and $h_{t-1}$ concatenated. So, we can select the appropriate elements of $ \frac{\partial{L}}{\partial{Z_t}} $ to make up $ \frac{\partial{L}}{\partial{X_t}} $ and $ \frac{\partial{L}}{\partial{h_{t-1}}} $.

# Data Flows Through Them

That explains how data flows through individual LSTM cells. Again, we feed in:

* A vector of size (`vocab_size`)
* The hidden state vector from the prior time step
* The cell state vector from the prior time step

We get out:

* A vector of size (`vocab_size`)
* The hidden state vector to be fed back into this node at the next time step
* The cell state vector to be fed back into this node at the next time step

Here's a visual of how this works in individual cells:

![](img/LSTM_1.png)

How can this be combined into a neural network architecture? First, here's a visual of how data is passed between just two cells in the network:

![](img/LSTM_2.png)

## Example on sequences of length 5

How would this work if we have multiple layers though? Let's look at a simple example below: suppose our goal was to model sequences of length 5. We could think of 

First, the first element of the sequence would be fed into the LSTM cell. As described above, this cell would send the (finish this)

![](img/lstm_data_flow_1.png)

The data could also flow "vertically", with each character flowing through all the layers before flowing through to the next character (see visual below)

![](img/lstm_data_flow_2.png)

Understanding this should easily help you see how data should flow _backwards_ through an LSTM-based network. This is often explained as a separate algorithm called the "Backpropagation Through Time Algorithm (BPTT)". Whether or not you think it worthy of that moniker, what is happening is simply this:

![](img/lstm_backward_1.png)

Again, we could equivalently flow the data through like this:

![](img/lstm_backward_2.png)

But, these are equivalent, and coding up the first one turns out to be easier.

# Coding it up

## Imports

In [1]:
import matplotlib.pyplot as plt
from IPython import display
plt.style.use('seaborn-white')
%matplotlib inline

from copy import deepcopy
from collections import deque

## Activations

In [2]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))


def dsigmoid(y):
    return y * (1 - y)


def tanh(x):
    return np.tanh(x)


def dtanh(y):
    return 1 - y * y


def softmax(x):
    return np.exp(x) / np.sum(np.exp(x)) #softmax

## `LSTM_Param` class

In [3]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))


def dsigmoid(y):
    return y * (1 - y)


def tanh(x):
    return np.tanh(x)


def dtanh(y):
    return 1 - y * ya


def softmax(x):
    return np.exp(x) / np.sum(np.exp(x)) #softmax

In [ ]:
class LSTM_Param:
    def __init__(self, value):
        self.value = value
        self.deriv = np.zeros_like(value) #derivative
        self.momentum = np.zeros_like(value) #momentum for AdaGrad
        
    def clear_gradient(self):
        self.deriv = np.zeros_like(self.value) #derivative
        
    def clip_gradient(self):
        self.deriv = np.clip(self.deriv, -1, 1, out=self.deriv)
        
    def update(self, learning_rate):
        self.momentum += self.deriv * self.deriv # Calculate sum of gradients
        self.value += -(learning_rate * self.deriv / np.sqrt(self.momentum + 1e-8))
        
    def update_sgd(self, learning_rate):
        self.value -= learning_rate * self.deriv